In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

In [2]:
os.chdir('..')

In [3]:
os.getcwd()

'd:\\onnx'

In [4]:
df = pd.read_csv('Dataset/preprocessed.csv')

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [6]:
df.shape

(674, 8)

In [7]:
x = df.drop('Survived', axis= 1).values
y = df['Survived'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state=42, shuffle= True, stratify= df['Survived'])

In [9]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((539, 7), (539,), (135, 7), (135,))

In [10]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [11]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((539, 7), (539, 1), (135, 7), (135, 1))

In [12]:
class data(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype= torch.float32)
        self.y = torch.tensor(y, dtype= torch.float32)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [13]:
train_ds = data(X_train, y_train)
test_ds = data(X_test, y_test)

In [14]:
train_loader = DataLoader(train_ds, batch_size= 16, shuffle= True)
test_loader = DataLoader(test_ds, batch_size=16)

In [15]:
class ANNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(X_train.shape[1], 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [17]:
model = ANNModel()
optimizer = Adam(model.parameters(), lr= 1e-4)
criterion = nn.BCEWithLogitsLoss()
model.to(device)

ANNModel(
  (net): Sequential(
    (0): Linear(in_features=7, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.4, inplace=False)
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [18]:
epochs = 20
for epoch in range(epochs):
    total_loss = 0
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

Epoch 1/20, Loss: 1.1678
Epoch 2/20, Loss: 1.0481
Epoch 3/20, Loss: 0.9851
Epoch 4/20, Loss: 0.8798
Epoch 5/20, Loss: 0.8408
Epoch 6/20, Loss: 0.9101
Epoch 7/20, Loss: 0.8750
Epoch 8/20, Loss: 0.9128
Epoch 9/20, Loss: 0.8232
Epoch 10/20, Loss: 0.8157
Epoch 11/20, Loss: 0.7622
Epoch 12/20, Loss: 0.7785
Epoch 13/20, Loss: 0.8230
Epoch 14/20, Loss: 0.7403
Epoch 15/20, Loss: 0.7302
Epoch 16/20, Loss: 0.7592
Epoch 17/20, Loss: 0.7457
Epoch 18/20, Loss: 0.7126
Epoch 19/20, Loss: 0.7290
Epoch 20/20, Loss: 0.7344


In [19]:
correct = 0
total = 0

model.eval() 
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        outputs = model(X_batch)
        probs = torch.sigmoid(outputs)
        preds = (probs >= 0.5).float()

        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.6889


In [20]:
torch.save(model.state_dict(), 'Models/ANN.pth')

In [24]:
import torch.onnx as onnx

dummy_input = torch.randn(1, X_train.shape[1])

model.to('cpu')

onnx.export(
    model,
    dummy_input,
    "Models/ANN.onnx",
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes= { 'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}}
)